<a href="https://colab.research.google.com/github/sergienko4/deep-learing/blob/main/ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning: Ex.2 - Multilayer Perceptron 

Submitted by: [... **name & ID** ...]


In [ ]:
# TensorFlow 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

---
In this exercise we will work with two datasets (see `class2.ipynb` for details): 
- **MNIST**: handwritten digits (0-9), each sample is an image of size 28x28 (without colors)
- **CIFAR-10**: color images of 10 classes (airplanes, birds, etc..), each sample is of size 32x32x**3**

We will try to fit different models for each of these datasets:
1. **0-hidden:** simplest model, without any hidden layers
2. **1-hidden:** a model with one hidden layer (with 32 neurons in that layer)
3. **2-hidden:** a model with two hidden layer (with 32 neurons in each of these hidden layers)


General instructions:

- For each model, the input layer is the images (after flatten it to long vector), and the output layer is a `softmax` of 10 units (one unit for each class).

- Activations: use `ReLU` for the hidden-layers, and `softmat` for the output layer.

- Display the `model.summary()` for each model.

- Train each model for **50 epochs** (use: `batch_size=64` and `verbose=0`).

- For each model, **plot the training loss & accuracy plots**.

- Summarize the results of all models (for both datasets) in the table below (by editing this markup cell):


<table>
  <tr>
    <th>Dataset</th>
    <th>Model</th>
    <th>#parameters</th>
    <th>train accuracy</th>
    <th>test accuracy</th>
  </tr>
    
  <!-- copy this block once for every model/dataset you tested -->  
  <tr> 
    <td>MNIST</td>   <!-- Dataset -->  
    <td>0-hidden</td>   <!-- Model -->
    <td>??? </td> <!-- #parameters -->
    <td>???</td> <!-- train accuracy -->
    <td>???</td> <!-- test accuracy -->
  </tr>
    
 
</table>

---



#### Loading the MNIST Dataset

In [ ]:
# 1. load/download the data
(MNIST_train_images, MNIST_train_labels), (MNIST_test_images, MNIST_test_labels) = tf.keras.datasets.mnist.load_data()

# 2. flatten the labels (easier to deal with)
MNIST_train_labels = MNIST_train_labels.flatten()  # (50000, 1) -> (50000,)
MNIST_test_labels = MNIST_test_labels.flatten()    # (10000, 1) -> (10000,)

# 3. convert uint8->float32 and normalize range to 0.0-1.0 
MNIST_train_images = MNIST_train_images.astype('float32') / 255.0
MNIST_test_images = MNIST_test_images.astype('float32') / 255.0

# 4. expand the dimensions (tensorflow expects images in a (H,W,C) format):
# (Height, Width, Channel), in our case we have 3 color channels (R,G,B)
MNIST_train_images = MNIST_train_images[...,None]
MNIST_test_images = MNIST_test_images[..., None]

# 5. print the shapes
print('MNIST_train_images.shape =',MNIST_train_images.shape)
print('MNIST_train_labels.shape =',MNIST_train_labels.shape)
print('MNIST_test_images.shape =',MNIST_test_images.shape)
print('MNIST_test_labels.shape =',MNIST_test_labels.shape)

# 6. lets plot some "5"s (just for fun)
[idx] = np.where(MNIST_train_labels==5) # find all the "5"s
plt.figure(figsize=(15,8))
for i in range(50):
    plt.subplot(5,10,i+1)
    plt.imshow(MNIST_train_images[idx[i],:,:,0], cmap='gray')
    plt.xticks([]), plt.yticks([])

---
#### Loading the CIFAR-10 Dataset

When running this command for the first time, it will download dataset from a remote server, which might take some time.. (in case of server error - just try again a bit later)

In [ ]:
# 1. load/download the data
(CIFAR_train_images, CIFAR_train_labels), (CIFAR_test_images, CIFAR_test_labels) = tf.keras.datasets.cifar10.load_data()

# 2. flatten the labels (easier to deal with)
CIFAR_train_labels = CIFAR_train_labels.flatten()  # (50000, 1) -> (50000,)
CIFAR_test_labels = CIFAR_test_labels.flatten()    # (10000, 1) -> (10000,)

# 3. convert uint8->float32 and normalize range to 0.0-1.0 
CIFAR_train_images = CIFAR_train_images.astype('float32') / 255.0
CIFAR_test_images = CIFAR_test_images.astype('float32') / 255.0

# 4. define the 10 classes names
CIFAR_class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

# 5. print the shapes
print('CIFAR_train_images.shape =',CIFAR_train_images.shape)
print('CIFAR_train_labels.shape =',CIFAR_train_labels.shape)
print('CIFAR_test_images.shape =',CIFAR_test_images.shape)
print('CIFAR_test_labels.shape =',CIFAR_test_labels.shape)

# 6. lets plot some 'dogs' (just for fun)
[idx] = np.where(CIFAR_train_labels==5) # find all the dogs
plt.figure(figsize=(15,8))
for i in range(50):
    plt.subplot(5,10,i+1)
    plt.imshow(CIFAR_train_images[idx[i]])
    plt.xticks([]), plt.yticks([])

***
#### 1. MNIST with 0-hidden


In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
#### 2. MNIST with 1-hidden

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
#### 3. MNIST with 2-hidden


In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
#### 4. CIFAR with 0-hidden


In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
#### 5. CIFAR with 1-hidden



In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
#### 6. CIFAR with 2-hidden


In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***

## Good Luck!

- **don't forget to fill the summary table on the top !!**
